# Lab 6 - Dependency parsing using machine learning techniques
Laboration 6 in EDAN20 @ LTH - http://cs.lth.se/edan20/coursework/assignment-6/

Author: Jonatan Kronander

## Objectives
The objectives of this assignment are to:
* Extract feature vectors and train a classifier
* Write a statistical dependency parser
* Understand how to design parameter sets
* Write a short report on your results

## Parsing the corpus and evaluating the results
First load model data from lab 5.

In [37]:
import pickle

with open('X_feature1.pickle', 'rb') as handle:
    X1 = pickle.load(handle)

with open('Y_feature1.pickle', 'rb') as handle:
    Y1 = pickle.load(handle)

with open('X_feature2.pickle', 'rb') as handle:
    X2 = pickle.load(handle)
    
with open('Y_feature2.pickle', 'rb') as handle:
    Y2 = pickle.load(handle)
    
with open('X_feature3.pickle', 'rb') as handle:
    X3 = pickle.load(handle)
    
with open('Y_feature2.pickle', 'rb') as handle:
    Y3 = pickle.load(handle)

#### Training the classifiers

In [38]:
feature_names_1 = ['pos_stack_0', 'word_stack_0',
                    'pos_queue_0','word_queue_0',
                    'can-re', 'can-la']

feature_names_2 = ['pos_stack_0', 'pos_stack_1', 'word_stack_0', 'word_stack_1',
                    'pos_queue_0', 'pos_queue_1', 'word_queue_0', 'word_queue_1',
                    'can-re', 'can-la']

feature_names_3 = ['pos_stack_0', 'pos_stack_1', 'word_stack_0', 'word_stack_1',
                    'pos_queue_0', 'pos_queue_1', 'word_queue_0', 'word_queue_1', 
                    'pos_next_stack', 'word_next_stack',
                    'pos_next_queue', 'pos_next_queue',
                    'can-re', 'can-la']

In [46]:
def create_feature_dict(X, feature_names):
    """
    Creates a feature dict from feature names and feature list
    :param: feature list
    :param: feature names
    :return: dict
    """
    X_train = []
    for el in X:
        X_dict = {key: el[i] for i,key in enumerate(feature_names)}
        X_train.append(X_dict)
        
    return X_train
    

In [47]:
X1_dict = create_feature_dict(X1, feature_names_1)
X2_dict = create_feature_dict(X2, feature_names_2)
X3_dict = create_feature_dict(X3, feature_names_3)

Training with features 1:

In [55]:
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer

print("Encoding the features...")
# Vectorize the feature matrix and carry out a one-hot encoding
vec = DictVectorizer(sparse=True)
X = vec.fit_transform(X1_dict)

print("Training the model...")
classifier = linear_model.LogisticRegression(penalty='l2', dual=True, solver='liblinear')
model1 = classifier.fit(X, Y1)
print(model1)

Encoding the features...
Training the model...
LogisticRegression(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


Training with features 2:

In [53]:
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer

print("Encoding the features...")
# Vectorize the feature matrix and carry out a one-hot encoding
vec = DictVectorizer(sparse=True)
X = vec.fit_transform(X2_dict)

print("Training the model...")
classifier = linear_model.LogisticRegression(penalty='l2', dual=True, solver='liblinear')
model2 = classifier.fit(X, Y2)
print(model2)

Encoding the features...
Training the model...
LogisticRegression(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


Training with features 3:

In [54]:
from sklearn import linear_model
from sklearn.feature_extraction import DictVectorizer

print("Encoding the features...")
# Vectorize the feature matrix and carry out a one-hot encoding
vec = DictVectorizer(sparse=True)
X = vec.fit_transform(X3_dict)

print("Training the model...")
classifier = linear_model.LogisticRegression(penalty='l2', dual=True, solver='liblinear')
model3 = classifier.fit(X, Y3)
print(model3)

Encoding the features...
Training the model...
LogisticRegression(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


#### Then we want to embed the data in Nivre's parser to compute their respective efficiencies.

In [2]:
import transition
import conll
import dparser

In [3]:
column_names_2006 = ['id', 'form', 'lemma', 'cpostag', 'postag', 'feats', 'head', 'deprel', 'phead', 'pdeprel']

sentences_train = conll.read_sentences("train.conll")
formatted_corpus_train = conll.split_rows(sentences_train, column_names_2006)

In [4]:
column_names_2006 = ['id', 'form', 'lemma', 'cpostag', 'postag', 'feats', 'head', 'deprel', 'phead', 'pdeprel']

sentences_test = conll.read_sentences("test_blind.conll")
formatted_corpus_test = conll.split_rows(sentences_test, column_names_2006)


#### We proceed sentence by sentence, and word by word. For a certain state, it will predict the next action using our classifier. We will then execute the corresponding action: la, ra, re, or sh. If an action is not possible, we will carry out a shift.